In [1]:
from scipy.spatial.transform import Rotation as R
import math
import numpy as np
import numba

q = np.random.rand(4)
q = q / np.linalg.norm(q)
q0, q1, q2, q3 = q

In [2]:
%%timeit
rotation = R.from_quat(q, scalar_first=True)
euler = rotation.as_euler("YXZ", degrees=True)
yaw, pitch, roll = euler

10.4 μs ± 172 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
rad2pi = 180 / math.pi

In [3]:
@numba.njit
def trans(q):
    rad2pi = 180 / math.pi
    q0, q1, q2, q3 = q
    q1_2 = q1**2
    q2_2 = q2**2
    q3_2 = q3**2
    yaw = math.atan2(
        2*(q0*q2 + q1*q3),
        1 - 2*(q2_2+q1_2)
    ) * rad2pi
    pitch = math.asin(2*(q0*q1 - q2*q3)) * rad2pi
    roll = math.atan2(
        2*(q0*q3 + q1*q2),
        1 - 2*(q1_2+q3_2)
    ) * rad2pi
    return yaw, pitch, roll

In [8]:
%%timeit
yaw, pitch, roll = trans(q)

182 ns ± 3.72 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
%%timeit
q0, q1, q2, q3 = q
q1_2 = q1**2
q2_2 = q2**2
q3_2 = q3**2
yaw = math.atan2(
    2*(q0*q2 + q1*q3),
    1 - 2*(q2_2+q1_2)
) * rad2pi
pitch = math.asin(2*(q0*q1 - q2*q3)) * rad2pi
roll = math.atan2(
    2*(q0*q3 + q1*q2),
    1 - 2*(q1_2+q3_2)
) * rad2pi

1.37 μs ± 39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
import timm
import urllib
import torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.models._efficientnet_blocks import UniversalInvertedResidual
from timm.layers.conv_bn_act import ConvNormAct
from rich import print
from torchinfo import summary
from PIL import Image
import torch.nn as nn

In [9]:
timm.list_models("*hrnet*")

['hrnet_w18',
 'hrnet_w18_small',
 'hrnet_w18_small_v2',
 'hrnet_w18_ssld',
 'hrnet_w30',
 'hrnet_w32',
 'hrnet_w40',
 'hrnet_w44',
 'hrnet_w48',
 'hrnet_w48_ssld',
 'hrnet_w64']

In [22]:
model = timm.create_model("hrnet_w18_ssld",
                          pretrained=False,
                          in_chans=1,
                          features_only=True)
shape = (480, 768)
i = torch.randn(1, 1, *shape)
o = model(i)
for j in o:
    print(j.shape)
print(model.feature_info.channels())
summary(model,
        input_size=(1, 1, *shape),
        col_names=("input_size", "output_size", "num_params", "params_percent", "mult_adds"),
        depth=3)

torch.Size([1, 64, 240, 384])

torch.Size([1, 128, 120, 192])

torch.Size([1, 256, 60, 96])

torch.Size([1, 512, 30, 48])

torch.Size([1, 1024, 15, 24])

[64, 128, 256, 512, 1024]

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Param %                   Mult-Adds
HighResolutionNetFeatures                               [1, 1, 480, 768]          [1, 64, 240, 384]         --                             --                   --
├─Conv2d: 1-1                                           [1, 1, 480, 768]          [1, 64, 240, 384]         576                         0.01%                   53,084,160
├─BatchNorm2d: 1-2                                      [1, 64, 240, 384]         [1, 64, 240, 384]         128                         0.00%                   128
├─ReLU: 1-3                                             [1, 64, 240, 384]         [1, 64, 240, 384]         --                             --                   --
├─Conv2d: 1-4                                           [1, 64, 240, 384]         [1, 64, 120, 192]         36,864                      0.34%                   849,346,5

In [4]:
print(model)

MobileNetV3Features(
  (conv_stem): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Hardswish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv_dw): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
        (bn2): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pwl): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
      (1): InvertedResidual(
        (conv_pw): Conv2d(24, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv_dw): Conv2d(72, 72, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=72, bias=False)
        (bn2): BatchNormAct2d(
          72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pwl): Conv2d(72, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (2): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(24, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv_dw): Conv2d(72, 72, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=72, bias=False)
        (bn2): BatchNormAct2d(
          72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(72, 24, kernel_size=(1, 1), stride=(1, 1))
          (act1): ReLU(inplace=True)
          (conv_expand): Conv2d(24, 72, kernel_size=(1, 1), stride=(1, 1))
          (gate): Hardsigmoid()
        )
        (conv_pwl): Conv2d(72, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNormAct2d(
          40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path)

In [4]:
model = timm.create_model("mobilenetv4_conv_medium", features_only=True,)
# print(model.feature_info.channels())
print(model)

MobileNetV3Features(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): EdgeResidual(
        (conv_exp): Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pwl): Conv2d(128, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
      (0): UniversalInvertedResidual(
        (dw_start): ConvNormAct(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
        (pw_exp): ConvNormAct(
          (conv): Conv2d(48, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
        )
        (dw_mid): ConvNormAct(
          (conv): Conv2d(192, 192, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=192, bias=False)
          (bn): BatchNormAct2d(
            192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
        )
        (se): Identity()
        (pw_proj): ConvNormAct(
          (conv): Conv2d(192, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
        (dw_end): Identity()
        (layer_scale): Identity()
        (drop_path): Identity()
      )
      (1): UniversalInvertedResidual(
        (dw_start): ConvNormAct(
          (conv): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=80, bias=False)
          (bn): BatchNormAct2d(
            80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
        (pw_exp): ConvNormAct(
          (conv): Conv2d(80, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
        )
        (dw_mid): ConvNormAct(
          (conv): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=160, bias=False)
          (bn): BatchNormAct2d(
            160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
        )
        (se): Identity()
        (pw_proj): ConvNormAct(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
        (dw_end): Identity()
        (layer_scale): Identity()
        (drop_path): Identity()
      )
    )
    (2): Sequential(
      (0): UniversalInvertedResidual(
        (dw_start): ConvNormAct(
          (

In [2]:
a = torch.rand(1, 3, 224, 224)
conv = ConvNormAct(3, 3, 3, stride=2, dilation=2)
b = conv(a)
print(b.shape)
print(conv)

torch.Size([1, 3, 112, 112])

ConvNormAct(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2), dilation=(2, 2), bias=False)
  (bn): BatchNormAct2d(
    3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): ReLU(inplace=True)
  )
)